<a href="https://colab.research.google.com/github/Murad042/00_tensorflow_fundamentals/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#in this notebook, we're going to cover some of the most fundamental concepts of tensors using Tensorflow

More specificially ,we're going to cover:

* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function(a way tp speed up your Python functions)
* Using GPUs with Tensorflow( or TPUs)
* Exercises to try for yourself

#Introduction to Tensors

In [1]:
#import tensorflow
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
#create tensors with tf.constant
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Constant 1-D Tensor from a python list.
tensor=tf.constant([1, 2, 3, 4, 5, 6])
tensor

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6], dtype=int32)>

In [4]:
# Or a numpy array
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6]])
tf.constant(a)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [5]:
#If dtype is specified, the resulting tensor values are cast to the requested dtype.
tf.constant([1, 2, 3, 4, 5, 6], dtype=tf.float64)

<tf.Tensor: shape=(6,), dtype=float64, numpy=array([1., 2., 3., 4., 5., 6.])>

In [6]:
#If shape is set, the value is reshaped to match. Scalars are expanded to fill the shape:
tf.constant(0, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 0]], dtype=int32)>

In [7]:
tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3])

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [8]:
#tf.constant  has no effect if an eager Tensor is passed as the value, it even transmits gradients:
v = tf.Variable([0.0])
with tf.GradientTape() as g:
    loss = tf.constant(v + v)
g.gradient(loss, v).numpy()

array([2.], dtype=float32)

In [9]:
#But, since tf.constant embeds the value in the tf.Graph this fails for symbolic tensors:
with tf.compat.v1.Graph().as_default():
  i = tf.compat.v1.placeholder(shape=[None, None], dtype=tf.float32)
  t = tf.constant(i)




TypeError: ignored

* tf.constant will always create CPU (host) tensors. In order to create tensors on other devices, 
* use tf.identity. (If the value is an eager Tensor, however, the tensor will be returned unmodified as mentioned above.)

In [ ]:
#Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

In [ ]:
#Create a vector
vector = tf.constant([10,10])
vector

In [ ]:
#check the dimension of a vector
vector.ndim

In [ ]:
#create a matrix (has more than 1 dimension)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

In [ ]:
matrix.ndim

In [ ]:
#Create another matrix
another_matrix = tf.constant([[10.,7.],
                              [7.,10.],
                              [9.,8.]],dtype = tf.float16)#specify the data type with the dtype parameter
another_matrix

In [ ]:
another_matrix.ndim

In [ ]:
#let's create a Tensor
tensor = tf.constant([[[1,2,3],
                       [2,3,4]],
                      [[5,6,7],
                       [9,8,7]],
                      [[4,5,6],
                       [9,8,7]]
                        
])
tensor

In [ ]:
tensor.ndim

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g wind speed and direction)
* Matrix: a 2-dimesional array of numbers
* Tensor: an n-dimensional a array of numbers(n can be any number)

### Creating  tensors with `tf.variable`

In [ ]:
# Create  same tensor with tf.variable as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor,unchangeable_tensor

In [ ]:
## Let's try change one of the elements in our changeable tensor
changeable_tensor[0]

In [ ]:
changeable_tensor[0] = 7
changeable_tensor

In [ ]:
## How about we try .assign()
changeable_tensor[0].assign(7)

In [ ]:
## Let's try chanfe our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

### Creating Random Tensors
Random Tensors are tensors of some arbitrary size which contain random numbers

In [ ]:
## Create two random (but the same )  tensors 
random_1 = tf.random.Generator.from_seed(42) # set seef for reproducibilty
random_1 = random_1.normal(shape = (3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape = (3,2))

## Are they equal?
random_1,random_2, random_1==random_2

### Shuffling the order of elements in a tensor

In [ ]:
### Shuffle a tensor ( valuable for when you want to shuffle your data  so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [4,5]])
### Shuffle our not_shuffled tensor
tf.random.shuffle(not_shuffled)


In [ ]:
tf.random.set_seed(42)#doesn't change => global level seed
tf.random.shuffle(not_shuffled,seed =42)## change operation level seed

In [ ]:
not_shuffled

### Other ways to create tensors

In [ ]:
### Create a tensor of all ones
tf.ones([10,7])

In [ ]:
### Create a tensor of all zeros
tf.zeros(shape=(3,4))

### Turn Numpy arrays into tensors
The main difference between NumPy arrays and Tensorflow tensors is that tensors can be run on a GPU(much faster than numerical computing) Otherwise They are very similar

In [ ]:
### You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1,25,dtype =np.int32)
numpy_A


#X = tf.constant(some_matrix) #capital for matrix or tensor
#y = tf.constant(vector) #non-capital for vector


In [ ]:
A = tf.constant(numpy_A)
A

In [ ]:
A = tf.constant(numpy_A,shape=(2,3,4))
A


In [ ]:
2*3*4##that is why we use 2,3,4

### Getting information from our tensors
When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [ ]:
### Create a  rank 4 tensor(4 dimensions)
rank4_tensors = tf.zeros(shape=(2,3,4,5))
rank4_tensors

In [ ]:
rank4_tensors[0]

In [ ]:
rank4_tensors.shape,rank4_tensors.ndim,tf.size(rank4_tensors)

In [ ]:
## Get various attributes of our tensors
print("Datatype of every elemet: ",rank4_tensors.dtype)
print("Number of dimension(rank): ",rank4_tensors.ndim)
print("Shape of tensor: ",rank4_tensors.shape)
print("Elements along the 0 axis: ",rank4_tensors.shape[0])
print("Elements along the last axis: ",rank4_tensors.shape[-1])
print("Total number of element in our tensor: ",tf.size(rank4_tensors))
print("Total number of element in our tensor: ",tf.size(rank4_tensors).numpy())

### Indexing Tensors
Tensors can be indexed just like Python lists

In [ ]:
### Get first 2 elements of each dimenions
rank4_tensors[:2,:2,:2,:2]

In [ ]:
# Get the first element each dimension from each index except for final one
rank4_tensors[:1,:1,:1,:]

In [ ]:
rank4_tensors[:1,:,:1,:1]

In [ ]:
## Create a rank 2 tensor (2 dimesion)
rank4_tensors = tf.reshape(rank4_tensors,shape=(30,4))
rank4_tensors

In [ ]:
## Get a rank 2 tensor(2 dimension)
rank2_tesnors = tf.constant([[10,7],
                             [7,10]])
rank2_tesnors.shape, rank2_tesnors.ndim

In [ ]:
## Get the last item of each of row of our rank 2 tensor
rank2_tesnors[0][-1],rank2_tesnors[1][-1]##instead

In [ ]:
##instead
rank2_tesnors[:,-1]

In [ ]:
## Add in extra dimension to our rank 2 tensor
rank3_tensor = rank2_tesnors[...,tf.newaxis]
rank3_tensor

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank2_tesnors,axis=-1)#axis = -1 means expand the final axis

In [ ]:
tf.expand_dims(rank2_tesnors,axis=0)

### Manipulating tensors (tensor operations)
**Basic operation**

`+`,`-`,`*`,`/`

In [ ]:
## You can also add values a tensor using addition operaation
tensor = tf.constant([[10,7],[3,4]])
tensor + 10


In [ ]:
#Original tensor is unchanged
tensor

In [ ]:
tensor = tensor +10
tensor

In [ ]:
#Multiplication also works
tensor*10

In [ ]:
#Substraction if you want
tensor -20

In [ ]:
#We can use tensorflow built-in functions too
tf.multiply(tensor,10)

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operation

There are two rules our tensors (or matrices) to fulfill if we're going to matrix lutiply them
> The inner dimensions must match

> The resulting matrix has shape of the outer dimensions

In [ ]:
##Matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor,tensor)

In [ ]:
tensor*tensor#because of elemet wise operation

In [ ]:
#Matrix multiplication with Python operator '@'
tensor @tensor

In [ ]:
## Create a tensor (3,2) tensor
X = tf.constant([[2,3],
                 [3,4],
                 [5,6]])
y = tf.constant([[7,8],
                 [6,4],
                 [9,4]])
X,y


In [ ]:
## let's try to matrix multiply tensors of same shape
X @ y

In [ ]:
tf.matmul(X,y)

In [ ]:
## Let's change the shape of y
tf.reshape(y,shape=(2,3))

In [ ]:
# Try to matrix multiply X with reshaped y
tf.matmul(X,tf.reshape(y,shape=(2,3)))

In [ ]:
# try change shape of X instead of y
tf.matmul(tf.reshape(X,shape=(2,3)),y)

In [ ]:
# Can do the same with transpose
X,tf.transpose(X),tf.reshape(X,shape=(2,3))

In [ ]:
# Try matrix multiplication with transpose rather than reshape
tf.transpose(X),y,tf.matmul(tf.transpose(X),y)#transpose flips the Access's rather than shuffles around the elements of a tensorflow.

**The dot product**

Matrix multiplication is also referred to as the dot product

You can perform matrix multiplication using:

* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [ ]:
X,y

In [ ]:
## Perform the dot product on X and y (requires X or y to be transpose)
tf.tensordot(tf.transpose(X),y,axes=1)

In [ ]:
## Perform matrix multiplication between X and y(transposed)
tf.matmul(X,tf.transpose(y)),tf.tensordot(X,tf.transpose(y),axes=1)

In [ ]:
## Perform matrix multiplication between X and y(reshaped)
tf.matmul(X,tf.reshape(y,shape=(2,3))),tf.tensordot(X,tf.reshape(y,shape=(2,3)),axes=1)

In [ ]:
## Check the values of y, reshape y and transpose y
print('Normal y:')
print(y,"\n")

print("y rehsped to (2,3): ")
print(tf.reshape(y,shape=(2,3)),'\n')

print("y transposed: ")
print(tf.transpose(y))

In [ ]:
  tf.matmul(X,tf.transpose(y))

Generally , when performing matrix multiplication on two tensors and on of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors  to get satisfy the matrix mutiplication rules 

# Changing the datatype of tensors

In [ ]:
# Create a new tesnor with default datatype(int32)
B= tf.constant([1.7,3.4])
B.dtype

In [ ]:
C = tf.constant([7,10])
C.dtype

In [ ]:
# Change from float32 to float16
B = tf.cast(B,dtype=tf.float16)
B,B.dtype

But if we had a Tensor of a million elements and we reduced the floating point size from 32 to 16, we've basically halved the amount of space our Tensors is taking up on memory, allowing a hardware accelerator
to make calculations on it potentially twice as fast.

In [ ]:
# Change from int32 to float32
E = tf.cast(C,dtype=tf.float32)
E,E.dtype

In [ ]:
E_float16 = tf.cast(E,dtype=tf.float16)
E_float16,E_float16.dtype

#Aggregating the tensors

> Aggregating tensors = condensing them from multiple values down to a smallar amount of values

In [ ]:
# Get the absolute value
D = tf.constant([-7,-8])
D

In [ ]:
# Get the absolute values
tf.abs(D)

# Let's go through the following aggregating:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor



In [ ]:
# Create a random tensor with values between 0 and 100 of 50 size
E = tf.constant(np.random.randint(0,100,size=50))
E

In [ ]:
tf.size(E),E.ndim,E.shape

In [ ]:
# Find the minimum
tf.reduce_min(E)

In [ ]:
# Find the maximum 
tf.reduce_max(E)

In [ ]:
#Find the mean of tensor
tf.reduce_mean(E)

In [ ]:
# Find the sum of tensor
tf.reduce_sum(E)

In [ ]:
# Find the the variance of our tensors
import tensorflow_probability as tfp
tfp.stats.variance(E)

In [ ]:
# Find the standard deviation of our tensors
tf.math.reduce_std(tf.cast(E,dtype=tf.float32))

## Find the postional maximum and minumum

>you're going to see this a lot when your neural network outputs prediction probabilities, which we haven't seen yet.


In [ ]:
# Create a new tensor for finding positional maxiumum and minimum
tf.random.set_seed(42)
F=tf.random.uniform(shape=[50])
F

In [ ]:
## Find the positional maxiumum
tf.argmax(F)

In [ ]:
# Index on our largest value position
F[tf.argmax(F)]

In [ ]:
# Find the max value of F
tf.reduce_max(F)

In [ ]:
# Check for equality
assert F[tf.argmax(F)] == tf.reduce_max(F)

In [ ]:
 F[tf.argmax(F)] == tf.reduce_max(F)

In [ ]:
# Find the positional minimum
tf.argmin(F)

In [ ]:
## Index on our lowest value postion
F[tf.argmin(F)]

### Squeezing a tensor(or remove all single dimensions)

In [ ]:
## Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]),shape=(1,1,1,1,50))
G

In [ ]:
G.shape

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed

## One-hot encdoing tensors

> One hot encoding is a form of numerical encoding.

In [ ]:
# Create a list of indices
some_list = [0,1,2,3]#in another word red,blue,green,purple

# One hot encode our indices
tf.one_hot(some_list,depth=4)

In [ ]:
# Specify custom values for one hot encoding
tf.one_hot(some_list,depth = 4, on_value='yes',off_value='no')

# Squaring,log,square root

In [10]:
# Create a new tensor
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [11]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [15]:
# Find the square root
tf.sqrt(tf.cast(H,dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.99999994, 1.4142134 , 1.7320508 , 1.9999999 , 2.236068  ,
       2.4494896 , 2.6457512 , 2.8284268 , 3.        ], dtype=float32)>

In [19]:
# Find the log
tf.math.log(tf.cast(H,dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

#Tensors and Numpy

> Tensorflow interacts beautifully with Numpy array



In [20]:
# Create a tensor directly from Numpy array
J = tf.constant(np.array([1.,2.,2.,3.]))
J

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([1., 2., 2., 3.])>

In [21]:
# Convert our tensor back to a Numpy array
np.array(J),type(np.array(J))

(array([1., 2., 2., 3.]), numpy.ndarray)

In [23]:
# Convert tensor J to a Numpy array
J.numpy(),type(J.numpy())

(array([1., 2., 2., 3.]), numpy.ndarray)

In [25]:
J = tf.constant([3.])
J.numpy(),J.numpy()[0]

(array([3.], dtype=float32), 3.0)

In [26]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3.,4.,5.]))
tensor_j = tf.constant([3.,4.,5.])

# Check the default types
numpy_J.dtype,tensor_j.dtype

(tf.float64, tf.float32)

## The main difference between Tensorflow and Numpy ,Tensorflow can be run a GPU(or TPU) for faster numerical processing

##Finding access to GPUs

In [27]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [28]:
tf.config.list_physical_devices("GPU")

[]

In [2]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!nvidia-smi

Sun May 30 23:58:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Note that: If you have access to Cuda-enabled GPU,Tensorflow will automatically use it whenever posssible